In [142]:
import numpy as np
import pandas as pd
import geopandas as gpd
import psrcelmerpy

In [143]:
eg_conn = psrcelmerpy.ElmerGeoConn()

# Load network shapefile
gdf = gpd.read_file(r'R:\e2projects_two\SoundCast\Inputs\dev\networks\2023\network_2023_v2\shapefiles\AM\AM_edges.shp')

In [146]:
# Load bike stress table, adapted from Lowry
# https://www1.coe.neu.edu/~pfurth/Furth%20papers/2016%20Prioritizing%20to%20improve%20low-stress%20network%20connectivity%20Lowry,%20Furth,%20Hadden-Loh.pdf

df_data = pd.read_csv('bike_stress_table.csv')

In [148]:
# Reformat speed limit and lanes to match the bike stress data
# Creating a new column for speed limits where minimum is capped at 25 and max at 35
gdf['SpeedLimit_new'] = gdf['ul2'].copy()
gdf.loc[gdf['ul2'] <= 25, 'SpeedLimit_new'] = 25
gdf.loc[gdf['ul2'] >= 35, 'SpeedLimit_new'] = 35

# Set maximum lanes at 6 and minimum at 2

gdf['lanes_new'] = gdf['lanes'].copy().astype('int')
gdf.loc[gdf['lanes'] >= 6, 'lanes_new'] = 6
gdf.loc[gdf['lanes'] <2, 'lanes_new'] = 2

# Lowry has different values for neighborhood street for each bike facility type
# Create a flag for an local street
gdf['LocalStreet'] = 0
gdf.loc[gdf['FacilityTy'] == 9, 'LocalStreet'] = 1
# gdf['FacilityType'].unique()

In [149]:
gdf['FacilityTy'] = gdf['FacilityTy'].astype('int')
gdf['FacilityTy'].unique()

# Remove centroid connectors and freeways
facility_type_lookup = {
    1:'Freeway',   # Interstate
    2:'Freeway',   # Ohter Freeway
    3:'Freeway', # Expressway
    4:'Ramp',
    5:'Arterial',    # Principal arterial
    6:'Arterial',    # Minor Arterial
    7:'Collector',    # Major Collector
    8:'Collector',    # Minor Collector
    9:'Collector',   # Local
    10:'Busway',
    11:'Non-Motor',
    12:'Light Rail',
    13:'Commuter Rail',
    15:'Ferry',
    16:'Passenger Only Ferry',
    17:'Connector',    # centroid connector
    18:'Connector',    # facility connector
    19:'HOV',    # HOV Only Freeway
    20:'HOV'    # HOV Flag
    }

gdf = gdf[gdf['FacilityTy'].isin([5,6,7,8,9,11])]

In [150]:
# Intersect with urban growth area
gdf_growth_area = eg_conn.read_geolayer('urban_growth_area')
gdf_growth_area = gdf_growth_area[['county_name','geometry']].to_crs('EPSG:2285')

gdf = gpd.sjoin(gdf, gdf_growth_area, how="left")

gdf['rural'] = 'not_rural'
gdf.loc[gdf['county_name'].isnull(),'rural'] = 'rural'

In [151]:
# Define transRefEdges facility types to match the study definitions

bike_type_map = {
    0: 'NoBikeFacility',    # No Bike Lane
    1: 'BikeLane',    # Striped Bike Lane
    2: 'ProtectedBikeLane',    # Protected Bike Lane
    3: 'NoBikeFacility',    # Paved Shoulder
    4: 'Sharrows',    # Shared Lane Markings
    5: 'NoBikeFacility',    # Bike Provision Undefined
    6: 'NoBikeFacility',    # Bike Provision Undefined
    8: 'SharedUsePath',    # Shared Use Path
    9: 'BufferedBikeLane',    # Buffered Bike Lane 
    10: 'NoBikeFacility',     # Neighborhood Greenway
    
}

# Create a mapping of local vs other area type
area_type_map = {
    0: 'Other',
    1: 'Neighborhood'
}

# Create new fields that use the bike facility and area type mapping
gdf['BikeFacility'] = gdf['bkfac'].map(bike_type_map)
gdf['AreaType'] = gdf['LocalStreet'].map(area_type_map)

# We will follow this designation for any local street with speeds 30 mph or less and 2 lanes (1 in each direction)
# If speed limit > 30, set area type to non-neighborhood
gdf.loc[(gdf['SpeedLimit_new']>30, 'AreaType')] = 'Other'

# If number of lanes > 2, set area type to non-neighborhood
# FIXME: should probably just be 1 lane in each direction
gdf.loc[(gdf['lanes_new']>2, 'AreaType')] = 'Other'

# If in rural area, treat paved shoulder as bike lanes
gdf.loc[((gdf['rural']=='rural')&(gdf['bkfac']==3)), 'BikeFacility'] = 'BikeLane'

# Merge to df_data to get stress values
gdf = gdf.merge(df_data, left_on=['SpeedLimit_new', 'lanes_new', 'BikeFacility','AreaType'], 
          right_on=['SpeedLimit', 'Lanes', 'BikeFacility', 'AreaType'], how='left')

In [152]:
# add impact of slope
gdf.loc[(gdf['upslp'] > .02) & (gdf['upslp'] <= .04), 'Factor'] = gdf['Factor'] + 0.37
gdf.loc[(gdf['upslp'] > .04) & (gdf['upslp'] <= .06), 'Factor'] = gdf['Factor'] + 1.2
gdf.loc[(gdf['upslp'] > .06), 'Factor'] = gdf['Factor'] + 3.2

In [153]:
# Assign LTS based on stress factor
gdf.loc[gdf['Factor'] < 0.1, 'LTS'] = 1
gdf.loc[(gdf['Factor'] >= 0.1) & (gdf['Factor'] < 0.3), 'LTS'] = 2
gdf.loc[(gdf['Factor'] >= 0.3) & (gdf['Factor'] < 0.6), 'LTS'] = 3
gdf.loc[gdf['Factor'] >= 0.6, 'LTS'] = 4

In [155]:
gdf.to_file('2023_network_bike_stress.shp')

c:\Users\Modeller\AppData\Local\anaconda3\envs\summary\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.
